<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pip
!pip install pycountry
import pycountry
!pip install https://github.com/elyase/geotext/archive/master.zip
from geotext import GeoText
from google.colab import files

%pylab inline

url='https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

tweets = pd.read_csv(url)
test = pd.read_csv(url_test)
tweets.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
  Using cached https://github.com/elyase/geotext/archive/master.zip
  Created wheel for geotext: filename=geotext-0.3.0-py2.py3-none-any.whl size=2272453 sha256=a335d12094b4cae324c0ce267f8b1e94eb1b7cd9fcfbed6a21fc124b58710e1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-04dgkb6f/wheels/f5/e3/84/31638877059a434d8601a764fc7565f2a9f7b6fb327085191e
Successfully built geotext
Populating the interactive namespace from numpy and matplotlib


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test['character_count'] = test['text'].apply(len)
tweets['character_count'] = tweets['text'].apply(len)
tweets.head()

,id,keyword,location,text,target,character_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88


In [5]:
def word_count(text):
  return len(text.split())
  
tweets['word_count'] = tweets['text'].apply(word_count)

test['word_count'] = test['text'].apply(word_count)
tweets.head()

,id,keyword,location,text,target,character_count,word_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16


In [6]:
def cw_fire(text):
  returnable = 0
  if (text.find('fire') != -1):
    returnable = 1
  return returnable

tweets['contains_word_fire'] = tweets['text'].apply(cw_fire)

test['contains_word_fire'] = test['text'].apply(cw_fire)
tweets.head()

,id,keyword,location,text,target,character_count,word_count,contains_word_fire
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1


In [7]:
def cw_storm(text):
  returnable = 0
  if (text.find('storm') != -1):
    returnable = 1
  return returnable

def cw_flood(text):
  returnable = 0
  if (text.find('flood') != -1):
    returnable = 1
  return returnable

def cw_death(text):
  returnable = 0
  if (text.find('death') != -1):
    returnable = 1
  return returnable

def cw_love(text):
  returnable = 0
  if (text.find('love') != -1):
    returnable = 1
  return returnable

tweets['contains_word_storm'] = tweets['text'].apply(cw_storm)
tweets['contains_word_flood'] = tweets['text'].apply(cw_flood)
tweets['contains_word_death'] = tweets['text'].apply(cw_death)
tweets['contains_word_love'] = tweets['text'].apply(cw_love)
test['contains_word_storm'] = test['text'].apply(cw_storm)
test['contains_word_flood'] = test['text'].apply(cw_flood)
test['contains_word_death'] = test['text'].apply(cw_death)
test['contains_word_love'] = test['text'].apply(cw_love)
tweets.head()

,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0


In [8]:
tweets['has_mentions'] = tweets['text'].str.contains('@')
tweets['amount_of_exclamation_marks'] = tweets['text'].str.count('\!')
tweets['amount_of_interrogation_marks'] = tweets['text'].str.count('\?')
tweets['has_link'] = (tweets['text'].str.contains('http://') | tweets['text'].str.contains('https://') )
test['has_mentions'] = test['text'].str.contains('@')
test['amount_of_exclamation_marks'] = test['text'].str.count('\!')
test['amount_of_interrogation_marks'] = test['text'].str.count('\?')
test['has_link'] = (test['text'].str.contains('http://') | test['text'].str.contains('https://') )
tweets.iloc[30:40]

,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link
30,44,NaN,NaN,The end!,0,8,2,0,0,0,0,0,False,1,0,False
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,55,5,0,0,0,0,0,True,0,0,True
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,67,10,0,0,0,0,0,False,0,0,True
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,82,9,0,0,0,0,0,False,0,0,True
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,34,7,0,0,0,0,0,False,1,0,False
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,76,13,0,0,0,0,0,False,0,0,True
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0,122,21,0,0,0,0,0,True,0,0,False
37,55,ablaze,World Wide!!,INEC Office in Abia Set Ablaze - http://t.co/3...,1,55,8,0,0,0,0,0,False,0,0,True
38,56,ablaze,NaN,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ...,1,140,19,0,0,0,0,0,False,0,0,True
39,57,ablaze,Paranaque City,Ablaze for you Lord :D,0,22,5,0,0,0,0,0,False,0,0,False


In [9]:
# Esto demora bastantes segundos
stopwordsList = stopwords.words('english')
tweets['stopwords'] = ' '
tweets['stopwords'] = tweets['stopwords'].astype(str)
tweets['stopwords_count'] = 0
tweets.loc[:,'stopwords'] = '|'.join(word for word in (tweets.loc[:,'text']).to_string().split() if word in stopwordsList)
tweets.loc[:,'stopwords_count'] = tweets.loc[:,'text'].str.count(tweets.loc[:,'stopwords'].to_string())
test['stopwords'] = ' '
test['stopwords'] = test['stopwords'].astype(str)
test['stopwords_count'] = 0
test.loc[:,'stopwords'] = '|'.join(word for word in (test.loc[:,'text']).to_string().split() if word in stopwordsList)
test.loc[:,'stopwords_count'] = test.loc[:,'text'].str.count(test.loc[:,'stopwords'].to_string())
test = test.drop(['stopwords'], axis=1)

tweets.head()

,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords,stopwords_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,7
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,3
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,13
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,2
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,10


In [10]:
def countries(text):
   for country in pycountry.countries:
     if country.name in text:
       return True; 
   return False

def cities (text):
   cities = GeoText(text)
   if len(cities.cities) == 0:
     return False
   return True     

tweets['contains_countries']=tweets['text'].apply(countries)  
tweets['contains_cities']=tweets['text'].apply(cities)

test['contains_countries']=test['text'].apply(countries)  
test['contains_cities']=test['text'].apply(cities)
tweets.head(5)    


,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords,stopwords_count,contains_countries,contains_cities
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,7,False,False
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,3,True,False
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,13,False,False
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,2,False,False
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,are|the|of|this|to|in|are|this|from|as|on|of|t...,10,False,False


In [11]:
tweets.to_csv('train-features.csv')
test.to_csv('test-features.csv')
#files.download('train-features.csv')